In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pylab as plt
import seaborn as sns
from sklearn import linear_model
import seaborn as sns

pd.set_option('display.max_columns', None)  # 最多所有列
pd.set_option('display.max_rows', None)  # 显示所有行
# pd.set_option('display.float_format',lambda x: '%.6f'%x) # #显示小数位数两位
pd.set_option('display.width', 1024)  # 显示宽度
# pd.set_option('precision', 1)   # 设置显示数值的精度

import warnings
warnings.filterwarnings('ignore')  # 关闭运行时的警告
np.set_printoptions(linewidth=100, suppress=True)   # 打印numpy时设置显示宽度，并且不用科学计数法显示
print('载入完成')

#无风险收益率
s = 2.12/100
#交易日设定为252
#每日无风险利率为
r_s = np.power((1.+s), (1./252))- 1.
print(r_s)

载入完成
8.325110956697301e-05


In [96]:
start_date = '2019-01-01'  # 回测开始时间
end_date = '2021-08-13'  # 结束时间

instruments =  ['159995.ZOF','159928.ZOF','512700.HOF','512880.HOF','512690.HOF','512010.HOF','512680.HOF','512980.HOF','515700.HOF','512400.HOF','512580.HOF','515050.HOF','159825.ZOF','588000.HOF','159870.ZOF','513100.HOF','513050.HOF','159819.ZOF','512200.HOF','159996.ZOF','159920.ZOF','162411.ZOF']  # 

# 获取数据
data = DataSource('bar1d_CN_FUND').read(instruments, (pd.to_datetime(start_date) - datetime.timedelta(days=60)).strftime('%Y-%m-%d'), end_date, fields=['close'])  # 基于回测结束日期向前多取200天数据

# 整理数据
data = pd.pivot_table(data, values='close', index=['date'], columns=['instrument'])
# print(data)

# 处理复权数据
for i in range(len(instruments)):
    if instruments[i] == '512690.HOF':
        data.loc[:'2021-05-14', '512690.HOF'] = data.loc[:'2021-05-14', '512690.HOF'] / 1.961  # 512690.HOF 2021-05-14 之前数据 / 1.961
    elif instruments[i] == '512010.HOF':
        data.loc[:'2021-06-25', '512010.HOF'] = data.loc[:'2021-06-25', '512010.HOF'] / 3.905  # 512010.HOF 2021-06-25 之前数据 / 3.905
    elif instruments[i] == '159928.ZOF':
        data.loc[:'2021-06-24', '159928.ZOF'] = data.loc[:'2021-06-24', '159928.ZOF'] / 4  # 159928.ZOF 2021-06-24 之前数据 / 4

name_list = data.columns.to_list()
for i in range(len(name_list)):
    df_info = DataSource('basic_info_CN_FUND').read(name_list[i])  # 查询基金的基本信息
    if name_list[i] in ['159870.ZOF']:
        name_list[i] = '化工ETF'
    else:
        name_list[i] = df_info.loc[0, 'display_name']

t_data = data.copy()
t_data.columns=name_list
T.plot(t_data, options={'chart':{'height':800}})

In [97]:
t_data = t_data.pct_change()[1:]  # 收益率
t_data.fillna(0, inplace=True)
t_data.tail()

,AI智能,农业ETF,化工ETF,恒生ETF,消费ETF,芯片ETF,家电ETF,华宝油气,医药ETF,房地产,有色金属,环保ETF,军工基金,酒ETF,银行基金,证券ETF,传媒ETF,中概互联,纳指ETF,5GETF,新能车,科创50
date,,,,,,,,,,,,,,,,,,,,,,
2021-08-09,0.010805,0.030702,0.010120,0.003644,0.034807,-0.027165,0.027861,-0.010025,0.006720,0.036061,-0.020000,-0.024789,-0.005844,0.022452,0.025907,0.020814,0.022624,0.000000,-0.004967,-0.004006,-0.029224,-0.011515
2021-08-10,0.006803,0.003191,-0.014572,0.009441,0.038182,-0.005817,0.007326,0.012658,0.013351,-0.002677,-0.007741,-0.008666,0.037472,0.064189,0.018519,0.016682,0.019174,0.027112,0.004593,-0.004023,-0.020696,0.009197
2021-08-11,-0.007722,-0.007423,0.002773,0.002158,-0.017513,0.005266,0.001454,0.012500,-0.021080,0.033557,0.017021,0.008741,-0.003541,-0.019048,0.009091,-0.009116,-0.002894,-0.007980,-0.004969,-0.011309,0.016010,-0.004860
2021-08-12,0.000000,-0.006410,0.005530,-0.005025,-0.020499,0.005239,-0.017429,-0.002469,-0.017497,-0.007792,0.020921,-0.001733,-0.027718,-0.017799,-0.004914,-0.007360,-0.001451,-0.010520,-0.002797,0.009804,0.002521,0.004884
2021-08-13,-0.027237,0.008602,0.017415,-0.006494,0.004550,-0.042270,-0.001478,-0.007426,-0.005479,-0.006544,0.010929,-0.009838,-0.005848,0.006590,0.001646,0.000000,-0.002907,-0.020638,0.003606,-0.030744,-0.005030,-0.020656


In [98]:
ts_list = t_data.columns.to_list()
t_alpha = pd.DataFrame(columns=ts_list, index=t_data.index.values)
t_beta = pd.DataFrame(columns=ts_list, index=t_data.index.values)
# print(t_alpha)

In [99]:
#基准收益率设置为HS300指数日间收益率
r_m = DataSource('bar1d_CN_FUND').read('510330.HOF',start_date=start_date,end_date=end_date, fields=['close'])
#基准相对于无风险利率的超额收益率为
r_m['alpha'] = r_m['close'].pct_change() - r_s
r_m.set_index('date', drop=True, inplace=True)
r_m = r_m[1:]
r_m.tail()

,instrument,close,alpha
date,,,
2021-08-09,510330.HOF,5.047,0.010934
2021-08-10,510330.HOF,5.113,0.012994
2021-08-11,510330.HOF,5.082,-0.006146
2021-08-12,510330.HOF,5.046,-0.007167
2021-08-13,510330.HOF,5.020,-0.005236


In [100]:
rolling_days = 20
for i in range(len(ts_list)):
    df_tmp = t_data.iloc[:, [i]].copy()
    df_tmp = df_tmp.merge(r_m['alpha'], on='date', how='left')
    lrm = df_tmp['alpha'].values
    lrp = df_tmp[ts_list[i]].values
    date = df_tmp.index.values
    ltmp = pd.DataFrame({'rm': lrm, 'rp': lrp, 'date': date})
    for j in range(rolling_days, len(ltmp)):
        tmp = np.array(ltmp.iloc[j-rolling_days:j, :2])
        t_beta.iloc[j, i], t_alpha.iloc[j, i], rvalue, pvalue, stderr = stats.linregress(tmp)
# print(t_beta.head(), '\n', t_alpha.head())

In [101]:
t_alpha.dropna(inplace=True)
t_alpha.index.names = ['date']
t_alpha = t_alpha.astype('float')
T.plot(t_alpha, options={'chart':{'height':800}}, title='基金池的Alpha')

In [102]:
t_beta.dropna(inplace=True)
t_beta.index.names = ['date']
t_beta = t_beta.astype('float')
T.plot(t_beta, options={'chart':{'height':800}}, title='基金池的Beta')